In [502]:
from urllib.request import urlopen
import urllib
from bs4 import BeautifulSoup
import re

class URL:
    def __init__(self,url):
        self.url = url        
        self.getparams = []
        self.postparams = []


#root_url = 

class ScrapedURLs:
    def __init__(self, root_url):
        self.root = root_url
        self.urls = {self.root}
        self.keywords = set()
        self.form_keywords = set()
        self.visited = set()
        self.forms = dict()
        self.get_urls = dict()
        self.get_keywords = set()
    

    def reset(self):
        self.urls = {self.root}
        self.keywords = set()
        self.form_keywords = set()
        self.visited = set()
        self.forms = dict()

    def scrape_all(self):
        if len(self.urls)<=len(self.visited):
            return self.output()
        else:
            temp_urls = set()
            temp_keywords = set()
            temp_visited = set()

            unvisited = self.urls-self.visited

            # update keywords
            for u in unvisited:
                fragments =u[7:].split("/")
                fragments =set(fragments)-{""}
                if len(fragments | self.keywords) > len(self.keywords):
                    temp_keywords.update(fragments)

            existing_keywords = temp_keywords | self.keywords

            # scrape
            for u in unvisited:
                fragments =u[7:].split("/")
                fragments =set(fragments)-{""}

                if len(fragments | self.keywords) > len(self.keywords):
                    print("Scraping from "+u)
                    url,get_url,form = self.scrape(u)
                    if url:
                        for i in url:
                            #if i.find('?') == -1:                        
                            _frag = i[7:].split("/")
                            _frag = set(_frag)-{""}

                            if len(_frag | existing_keywords) > len(existing_keywords):
                                existing_keywords.update(_frag)
                                r = i
                                if i[-1]=='/':
                                    r = i[:-1]
                                temp_urls.add(i)
                                
                    if get_url:
                        for i in get_url:
                            r= i 
                            if i[-1]=="/":
                                r = i[:-1]
                            _frag = r[7:].split("/")
                            m = _frag[-1]  
                            key = r[:(m.find('=')-len(m))]
                            if key not in self.get_keywords:

                                        self.get_keywords.add(key)

                                        self.get_urls.update({r:[]})
                            
                    if form:
                        for key in form:
                            r= key[0].rstrip('/') 
                            _frag = r[7:].split("/")

                            if _frag[-1] not in self.form_keywords:

                                        self.form_keywords.add(_frag[-1])
                                        self.forms.update({r:form[key]})

                    temp_visited.add(u)

            self.visited.update(temp_visited)
            self.urls.update(temp_urls)
            self.keywords.update(temp_keywords)

            return self.scrape_all()    
        
    def scrape(self,url):
        try:
            page = urlopen(url)
        except urllib.error.HTTPError as e:
            print('HTTPError: {}'.format(e.code))
            return [], [], {}
        html = page.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")
        urls = self.find_url(soup,url)
        good, get = self.filter_url(urls)
        forms = self.find_form(soup,url)
        return good, get, forms
    
    
    def find_url(self,soup,url):
        if url[-1]!='/':
            url = url+'/'

        links=soup.find_all("a")
        if links == []:
            return []
        
        sub_urls = []
        for link in links:
            
            i = link["href"]
            i = re.sub(r"\/*","",i)
            if re.match("http",i):
                pass
            elif re.match("mailto:",i):
                pass
            else:
                sub_urls.append(url+i)
        return sub_urls
    
    def filter_url(self,url_list):
        good_urls = []
        get_urls = []
        for i in url_list:   
            t = i.rstrip('/')
            _frag = t.split('/')[-1]
            if '?' in _frag and '=' in _frag:
                get_urls.append(t)
            else:
                good_urls.append(t)
        return good_urls, get_urls        
        
            
    def find_form(self,soup,url):
        if url[-1]!='/':
            url = url+'/'

        forms = soup.find_all('form')
        actions = {}

        for form in forms:
             
            try:
                actions[(url,form['action'])] = []
                
            except:
                print("no action on "+url)
                continue

            for element in form.find_all('input'):
                if re.search('name',  str(element)):
                    actions[(url,form['action'])] .append(element['name'])
                    
                    #print(element['name'])
        return actions   
    
    def output(self):
        output_list = []
        allurls = {**self.forms, **self.get_urls}
        for i in allurls:
            _frag = i.split('/')[-1]
            # get
            if '?' in _frag and '=' in _frag:
                #_frag.split('=')[-1]
                #i[:(i.rfind("="))]

                u = URL(i[:(i.rfind("?"))])
                s = i.rfind("?")+1
                e = - (len(i[s:]) - i[s:].find("="))
                tmp = i[s:e]
                u.getparams.append(tmp)
                output_list.append(u)
                
            # post
            else:
                u = URL(i)
                u.postparams = self.forms[i]
                output_list.append(u)
        return output_list
        

In [503]:
s = ScrapedURLs('http://testphp.vulnweb.com')

In [504]:
url_list = s.scrape_all()

Scraping from http://testphp.vulnweb.com
Scraping from http://testphp.vulnweb.com/userinfo.php
Scraping from http://testphp.vulnweb.com/login.php
Scraping from http://testphp.vulnweb.com/artists.php
Scraping from http://testphp.vulnweb.com/cart.php
Scraping from http://testphp.vulnweb.com/privacy.php
HTTPError: 404
Scraping from http://testphp.vulnweb.com/guestbook.php
Scraping from http://testphp.vulnweb.com/disclaimer.php
Scraping from http://testphp.vulnweb.com/AJAXindex.php
HTTPError: 404
Scraping from http://testphp.vulnweb.com/Mod_Rewrite_Shop
Scraping from http://testphp.vulnweb.com/categories.php
Scraping from http://testphp.vulnweb.com/hpp
Scraping from http://testphp.vulnweb.com/index.php
Scraping from http://testphp.vulnweb.com/userinfo.php/signup.php
HTTPError: 302
Scraping from http://testphp.vulnweb.com/artists.php/#
HTTPError: 404
Scraping from http://testphp.vulnweb.com/Mod_Rewrite_Shop/Detailsweb-camera-a4tech2
HTTPError: 404
Scraping from http://testphp.vulnweb.com/Mo

In [505]:
for i in url_list:
    print(i.url)
    print(i.getparams, i.postparams)
    print('-----------------------')

http://testphp.vulnweb.com
[] ['searchFor', 'goButton']
-----------------------
http://testphp.vulnweb.com/userinfo.php
[] ['uname', 'pass']
-----------------------
http://testphp.vulnweb.com/login.php
[] ['uname', 'pass']
-----------------------
http://testphp.vulnweb.com/artists.php
[] ['searchFor', 'goButton']
-----------------------
http://testphp.vulnweb.com/cart.php
[] ['searchFor', 'goButton']
-----------------------
http://testphp.vulnweb.com/guestbook.php
[] ['name', 'submit']
-----------------------
http://testphp.vulnweb.com/disclaimer.php
[] ['searchFor', 'goButton']
-----------------------
http://testphp.vulnweb.com/categories.php
[] ['searchFor', 'goButton']
-----------------------
http://testphp.vulnweb.com/index.php
[] ['searchFor', 'goButton']
-----------------------
http://testphp.vulnweb.com/artists.php/artists.php
['artist'] []
-----------------------
http://testphp.vulnweb.com/categories.php/listproducts.php
['cat'] []
-----------------------
http://testphp.vulnweb

In [507]:
c=ScrapedURLs("http://www.testfire.net")
url_list2 = c.scrape_all()

Scraping from http://www.testfire.net
Scraping from http://www.testfire.net/cgi.exe
HTTPError: 404
Scraping from http://www.testfire.net/subscribe.jsp
Scraping from http://www.testfire.net/login.jsp
Scraping from http://www.testfire.net/index.jsp
Scraping from http://www.testfire.net/swaggerindex.html
HTTPError: 404
Scraping from http://www.testfire.net/feedback.jsp
Scraping from http://www.testfire.net/survey_questions.jsp
Scraping from http://www.testfire.net/status_check.jsp


In [508]:
for i in url_list2:
    print(i.url)
    print(i.getparams, i.postparams)
    print('-----------------------')

http://www.testfire.net
[] ['query']
-----------------------
http://www.testfire.net/subscribe.jsp
[] ['query']
-----------------------
http://www.testfire.net/login.jsp
[] ['query']
-----------------------
http://www.testfire.net/index.jsp
[] ['query']
-----------------------
http://www.testfire.net/feedback.jsp
[] ['query']
-----------------------
http://www.testfire.net/survey_questions.jsp
[] ['query']
-----------------------
http://www.testfire.net/status_check.jsp
[] ['query']
-----------------------
http://www.testfire.net/index.jsp
['content'] []
-----------------------
http://www.testfire.net/default.jsp
['content'] []
-----------------------
http://www.testfire.net/subscribe.jsp/index.jsp
['content'] []
-----------------------
http://www.testfire.net/login.jsp/index.jsp
['content'] []
-----------------------
http://www.testfire.net/index.jsp/index.jsp
['content'] []
-----------------------
http://www.testfire.net/index.jsp/default.jsp
['content'] []
-----------------------
ht

In [464]:
page = urlopen("http://testphp.vulnweb.com/artists.php")
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [449]:
forms = soup.find_all('form')

In [ ]:
wfuzz -c -z file,./wordlist/Injections/SQL.txt -d "searchFor=FUZZ&goButton=go" --hc 404 http://testphp.vulnweb.com/search.php
wfuzz -z file,wordlist/others/common_pass.txt -d "uname=FUZZ&pass=FUZZ"  --hc 302 http://testphp.vulnweb.com/userinfo.php